In [ ]:
'''
URJC / GIA / Aprendizaje Automático 1 / Curso 23-24
'''
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler
pd.set_option("display.precision", 2)

# Ingeniería de características

Como ya sabemos, los ejemplos con los que vamos a aprender nuestras máquinas se pueden presentar en forma de tabla donde:
- cada **fila** es un **ejemplo**
- cada **columna** es un **atributo** que hemos medido o registrado de los ejemplos.

En ocasiones estos atributos "tal cual" son suficientes, pero en muchas otras no nos proporcionan la información suficiente para realizar la tarea.

En esos casos una opción es crear nuevas columnas a partir de las que tenemos con la esperanza de lograr mejores resultados.

En este cuaderno comenzaremos a utilizar Scikit-Learn.


Llamaremos **vector características** (*features*) al vector con el que describimos cada ejemplo de manera definitiva, es decir el vector que finalmente se utiliza para aprender la tarea.

El vector de características puede coincidir con los atributos, pero también puede ser menor o mayor.
- **Menor**, si hemos eliminado atributos porque no resultaban útiles.
- **Mayor**, si hemos añadido nuevas columnas calculadas a partir de las que había inicialmente.
- **Igual**, si nos quedamos exactamente con los atributos dados o si creamos exactamente tantas columnas como las que eliminamos.

## Aumento de la dimensionalidad

Aumentar la dimensionalidad significa añadir nuevas característicias calculadas a partir de las que tenemos.

A diferencia de la imputación iterativa, ahora no tenemos que calcular una función de regresión; sino que somos nosotros los que elegimos el modo en el que se construyen las características.

><u>Ejemplo</u>.&nbsp; &nbsp;
Un cierto conjunto de ejemplos tiene, entre otros, dos atributos: "Población" y "Superficie". <br>
A partir de ellos podemos construir una nueva característica "Densidad de población",
$$\small
\text{Densidad de población} = \frac{\text{Población}}{\text{Superficie}}.
$$

Estas características generadas por nosotros suelen ser buenas si tenemos experiencia o conocimientos en el problema.

Esto no siempre es así, e incluso aunque tengamos experiencia es posible que no logremos dar con la expresión matemática más apropiada para generar una nueva características suficientemente "potente".

Una solución es lanzar una batería de transformaciones sobre las características; y un modo de hacerlo "organizadamente" es creando todas las combinaciones de características de un cierto grado polinómico que nosotros elegimos.

> Por ejemplo, con 2 características $x_1,~x_2$, las combinaciones de grado 2 serían: $\{x_1^2,~ x_1x_2,~ x_2^2\}$<br>
Sin embargo, la única combinación que contiene interacciones entre características es $\{x_1x_2\}$

Para ello contamos con el método  `PolynomialFeatures` de la biblioteca `sklearn.preprocessing`.

En el siguiente código podemos ver como se utiliza.

In [ ]:
# Ejemplo de como hacer aumentado de dimensionalidad mediante
# características polinómicas.

degree = 2
interaction_only = False #Hace todas las combinaciones, si lo ponemos en True hace solo las nuevas

df = pd.DataFrame({'x1': [1, 2, 3], 'x2': [4, 5, 6], 'x3': [7, 8, 9]})

polyf = PolynomialFeatures(degree=degree,
                           interaction_only=interaction_only)

polyf.set_output(transform="pandas")

polyf.fit(df)
df_poly = polyf.transform(df)

#Hay un metodo .fit-transform, pero mejor haserlo así (no hacer fit con test, solo train)

print('Dataframe inicial:')
print(df)
print('\nDataframe aumentado:')
print(df_poly)


Dataframe inicial:
   x1  x2  x3
0   1   4   7
1   2   5   8
2   3   6   9

Dataframe aumentado:
     1   x1   x2   x3  x1^2  x1 x2  x1 x3  x2^2  x2 x3  x3^2
0  1.0  1.0  4.0  7.0   1.0    4.0    7.0  16.0   28.0  49.0
1  1.0  2.0  5.0  8.0   4.0   10.0   16.0  25.0   40.0  64.0
2  1.0  3.0  6.0  9.0   9.0   18.0   27.0  36.0   54.0  81.0


Sobre el código de arriba hay que hacer los siguientes comentarios:
- Hemos creado dos variables al inicio que son las que controlan la demostración.
  - `degree` es el máximo grado de los polinomios que resultan
  - `interaction_only` sirve para quitar las características que interactuan consigo mismas.
- `polyf` es un objeto de la clase `PolynomialFeatures`.<br>
Esto es una característica común a toda la biblioteca sklearn y muy importante. <br>
Como sabemos los objetos tienen atributos; es decir que llevan dentro información.<br>
Pero además contienen métodos. En concreto `fit` y `transform` que son, junto con `predict`, los tres métodos que necesitamos en el 100% de las ocasiones.
- Primero se utiliza `fit` para que el objeto transformador `polyf` "aprenda" lo que debe hacer sobre el dataframe que le pasamos como parámetro.<br>
Después se utiliza `transform` para llevar a cabo esa transformación.
 - <u>¿Por qué separado en dos pasos?</u> <br>
 Una vez que el objeto ha aprendido a hacer la tarea, podemos aplicar el método `transform` sobre cualquier otro dataframe, con la única restricción de que tenga las mismas columnas que aquel con el que hemos hecho `fit`. <br>
 ¡Esto ocurre precisamente con el conjunto de Test!
 - <u>¿Por qué no usar `fit_transform`, que hace los dos pasos en uno solo?</u> <br>
 Yo, particularmente, NUNCA lo uso para forzar a emplear `fit` SOLO con el conjunto de entrenamiento y `transform` con el de entrenamiento y el de test.
 - <u>¿Por qué aparece una columna de unos a la izquierda del dataframe transformado?</u><br>
 Una de las tareas más frecuentes en ML es la regresión. <br>
 Esta columna de unos facilita la tarea de calcular el  *término independiente* de un modelo<br>
 &nbsp;(lo entenderemos mejor cuando lleguemos a esa parte de la asignatura).<br>
 Se puede eliminar con la opción `include_bias=False`.

## Reducción de la dimensionalidad

Los siguientes cuadernos están integramente dedicados a estas técnicas.

## Transformaciones sin modificar la dimensionalidad

Normalmente también se considera que hacer modificaciones sobre un atributo, aunque no se cree otro nuevo, es parte de la ingeniería de características.<br>
Vamos a estudiar transformaciones lineales, es decir escalados. En concreto:
  - al intervalo unidad
  - al máximo en valor absoluto
  - Estandarización

### Escalado al intervalo unidad

Dada una columna $x$, donde $x_{\rm min}~$ y $x_{\rm max}$ son el valor mínimo y máximo alcanzados, entonces la siguiente formula escala todos los valores al intervalo $[0,1]$

$$x_{\rm esc} = \frac{x - x_{\rm min}}{x_{\rm max}-x_{\rm min}}$$

- Esta operación se debe hacer columna a columna ya que los valores máximo y mínimo de $x_i$ pueden ser diferentes a los de $x_j$.
- Como resultado tenemos todas las características a la misma escala.
- Podemos utilizar `sklearn.preprocessing.MinMaxScaler`



### Escalado al máximo en valor absoluto

Dada una columna $x$, la siguiente formula escala todos los valores de modo que:
- si todos son positivos, el valor máximo es $1$ y el mínimo es mayor que cero
- si todos son negativos, el valor mínimo es $-1$ y todos son menores que cero
- si hay positivos y negativos, todos quedan transformados dentro del intervalo $[-1,1]$.


$$x_{\rm esc} = \frac{x}{\max(|x|)}$$

- Esta operación se también se debe hacer columna a columna ya que el máximo del valor absoluto de los valores de dos columnas puede ser diferente.
- Podemos utilizar `sklearn.preprocessing.MaxAbsScaler`

### Estandarización

Dada una columna $x$, con media $\mu$ y desviación $\sigma$, entonces la siguiente fórmula *estandariza* todos los valores de dicha columna
$$x_{\rm std} = \frac{x - \mu}{\sigma}$$

- Estandarizar significa que tiene media cero y desviación unidad.
- Esto **NO** significa que hayamos convertido la distribución de la columna $x$ en una normal.
$$¿ ~~ x_{\rm std} \sim \mathcal{N}(0,1) ~~ ?~~ \Large \leftarrow\text{ ¡NO!}$$
- Podemos utilizar `sklearn.preprocessing.StandardScaler`


### Ejemplo

En la siguiente celda se utilizan estas técnicas para mostrar las diferencias entre ellas.

In [ ]:
# función para imprimir por pantalla
def print_info(df):
  print(df)
  print(f'Mïnimos:      ',df.min().tolist())
  print(f'Máximos:      ',df.max().tolist())
  print(f'Medias:       ',df.mean().tolist())
  print(f'Desviaciones: ',df.std().tolist())

# Dataframe de juguete
df = pd.DataFrame({
    'x1': [1, 4, 2, 5, 3],
    'x2': [10, 20, 50, 30, 40],
    'x3': [-1, -4, -2, -5, -3],
    'x4': [10, -20, -50, -30, 40]
})

print(f'---Antes de los cambios----')
print_info(df)

# Escalado al intervalo unidad
scalerUnit = MinMaxScaler().set_output(transform="pandas")
scalerUnit.fit(df) #Almacena max + min de cada col; podemos verlo con scalerUnit.__dict__
scaleUnit_df = scalerUnit.transform(df)

print(f'\n---Tras el escalado a [0,1] de cada columna----')
print_info(scaleUnit_df)

# Escalado al maximo de los valores absolutos
scalerMaxabs = MaxAbsScaler().set_output(transform="pandas")
scalerMaxabs.fit(df) #Almacena max de cada col
scalerMaxabs_df = scalerMaxabs.transform(df)

print(f'\n---Tras el escalado al maximo de los valores absolutos de cada columna----')
print_info(scalerMaxabs_df)

#  Estandarización
scalerStd = StandardScaler().set_output(transform="pandas")
scalerStd.fit(df) #Almacena media y desviación
scaleStd_df = scalerStd.transform(df)

print(f'\n---Tras la estandarización de cada columna----')
print_info(scaleStd_df)

---Antes de los cambios----
   x1  x2  x3  x4
0   1  10  -1  10
1   4  20  -4 -20
2   2  50  -2 -50
3   5  30  -5 -30
4   3  40  -3  40
Mïnimos:       [1, 10, -5, -50]
Máximos:       [5, 50, -1, 40]
Medias:        [3.0, 30.0, -3.0, -10.0]
Desviaciones:  [1.5811388300841898, 15.811388300841896, 1.5811388300841898, 35.35533905932738]

---Tras el escalado a [0,1] de cada columna----
     x1    x2    x3    x4
0  0.00  0.00  1.00  0.67
1  0.75  0.25  0.25  0.33
2  0.25  1.00  0.75  0.00
3  1.00  0.50  0.00  0.22
4  0.50  0.75  0.50  1.00
Mïnimos:       [0.0, 0.0, 0.0, 0.0]
Máximos:       [1.0, 1.0, 1.0, 1.0]
Medias:        [0.5, 0.5, 0.5, 0.4444444444444445]
Desviaciones:  [0.39528470752104744, 0.39528470752104744, 0.39528470752104744, 0.3928371006591931]

---Tras el escalado al maximo de los valores absolutos de cada columna----
    x1   x2   x3   x4
0  0.2  0.2 -0.2  0.2
1  0.8  0.4 -0.8 -0.4
2  0.4  1.0 -0.4 -1.0
3  1.0  0.6 -1.0 -0.6
4  0.6  0.8 -0.6  0.8
Mïnimos:       [0.2, 0.2, -1.0,

In [ ]:
scalerStd.__dict__

{'with_mean': True,
 'with_std': True,
 'copy': True,
 '_sklearn_output_config': {'transform': 'pandas'},
 'feature_names_in_': array(['x1', 'x2', 'x3', 'x4'], dtype=object),
 'n_features_in_': 4,
 'n_samples_seen_': 5,
 'mean_': array([  3.,  30.,  -3., -10.]),
 'var_': array([   2.,  200.,    2., 1000.]),
 'scale_': array([ 1.41421356, 14.14213562,  1.41421356, 31.6227766 ])}


#Ejercicios
##Ejercicio 1
Crea un dataframe de juguete o utiliza uno que ya tengas con el único requisito de que tenga más de 5 atributos.*texto en cursiva*

A continuación añádele dimensiones polinómicas de grado 2 pero SOLO de **tres** columnas que elijas, manteniendo todas las demás.


In [ ]:
df_juguete = pd.DataFrame({
    'a1': [1, 3, 4, 6, 3],
    'a2': [3, 5, 7, 10, 4],
    'a3': [6, 7, 3, 2, 2],
    'a4': [5, 6, 7, 8, 9],
    'a5': [-2, -3, -4, -5, -6],
    'a6': [3, 6, 7, 9, 5]
})
print("Data frame original:")
print(df_juguete)

select_columns = ['a1', 'a3', 'a5']
df_select= df_juguete[select_columns]
polyf = PolynomialFeatures(degree=2, interaction_only=True)

polyf.set_output(transform="pandas")

polyf.fit(df_select)
df_poly = polyf.transform(df_select)

df_juguete = pd.concat([df_juguete, df_poly.drop(columns=select_columns)], axis=1)


print(df_juguete)


Data frame original:
   a1  a2  a3  a4  a5  a6
0   1   3   6   5  -2   3
1   3   5   7   6  -3   6
2   4   7   3   7  -4   7
3   6  10   2   8  -5   9
4   3   4   2   9  -6   5
   a1  a2  a3  a4  a5  a6    1  a1 a3  a1 a5  a3 a5
0   1   3   6   5  -2   3  1.0    6.0   -2.0  -12.0
1   3   5   7   6  -3   6  1.0   21.0   -9.0  -21.0
2   4   7   3   7  -4   7  1.0   12.0  -16.0  -12.0
3   6  10   2   8  -5   9  1.0   12.0  -30.0  -10.0
4   3   4   2   9  -6   5  1.0    6.0  -18.0  -12.0


##Ejercicio 2

Crea características $\log (\cdot)~$ de cada una de las características del dataframe resultante del ejercicio anterior.

> *Al hacer $\log~$ estamos convirtiendo en números negativos los valores menores que uno, y en positivos más pequeños, cercanos al orden de magnitud, los valores mayores que uno.*

¿Qué problemas puede introducir esta transformación?

In [ ]:
import math
for col in df_juguete.columns:

    #Para añadir cols distintas:
    #df_juguete['log_' + col] = df_juguete[col].apply(lambda x: math.log(x) if x > 0 else x)

    #aplicar directamente
   df_juguete[col]=df_juguete[col].apply(lambda x: math.log(x) if x > 0 else x)

print(df_juguete)



     a1    a2    a3    a4  a5    a6    1  a1 a3  a1 a5  a3 a5
0  0.00  1.10  1.79  1.61  -2  1.10  0.0   1.79   -2.0  -12.0
1  1.10  1.61  1.95  1.79  -3  1.79  0.0   3.04   -9.0  -21.0
2  1.39  1.95  1.10  1.95  -4  1.95  0.0   2.48  -16.0  -12.0
3  1.79  2.30  0.69  2.08  -5  2.20  0.0   2.48  -30.0  -10.0
4  1.10  1.39  0.69  2.20  -6  1.61  0.0   1.79  -18.0  -12.0


***Problemas***:

**Manejo de valores negativos**: La función logaritmo no está definida para números negativos, por lo que si hay valores negativos en las características resultantes de las características polinómicas, la aplicación de la función logarítmica a esos valores resultará en un error o NaN (no un número).

2.**Valores cercanos a cero**: Los valores cercanos a cero producirán valores grandes o infinitos negativos después de aplicar la función logaritmo. Esto podría distorsionar las distribuciones de los datos y afectar negativamente el rendimiento de los modelos que dependen de la normalidad de los datos.

3.**Reducción de la interpretabilidad**: Aplicar la transformación logarítmica a las características puede hacer que el modelo resultante sea menos interpretable, ya que los efectos de las características se vuelven más difíciles de interpretar directamente.



##Ejercicio 3
Crea características $e^{(\cdot)}~$ de cada una de las características del dataframe resultante del ejercicio anterior.

> *Al hacer $\exp~$ estamos convirtiendo todos los valores en positivos.* <br>
*Además todos los que tengan un valor absoluto mayor que uno son agrandados, y los que tiene valor absoluto menor que uno empequeñecidos.*

In [ ]:
import math
for col in df_juguete.columns:

    #aplicar directamente
   df_juguete[col]=df_juguete[col].apply(lambda x: math.exp(x) if x > 0 else x)

print(df_juguete)

    a1    a2   a3   a4  a5   a6    1  a1 a3  a1 a5  a3 a5
0  0.0   3.0  6.0  5.0  -2  3.0  0.0    6.0   -2.0  -12.0
1  3.0   5.0  7.0  6.0  -3  6.0  0.0   21.0   -9.0  -21.0
2  4.0   7.0  3.0  7.0  -4  7.0  0.0   12.0  -16.0  -12.0
3  6.0  10.0  2.0  8.0  -5  9.0  0.0   12.0  -30.0  -10.0
4  3.0   4.0  2.0  9.0  -6  5.0  0.0    6.0  -18.0  -12.0


##Ejercicio 4

Descarga un dataframe de internet o utiliza uno que ya tengas con el único requisito de que tenga más de 5 atributos.
Separalo en dos:
- trainDF
- testDF

A continuación realiza ingeniería de características sobre trainDF.<br>
Debe incluir:
- Aumento de dimensionalidad, con:
  - características creadas por ti,
  - características polinómicas de orden 3, pero SOLO de 2 características que elijas y con la opción `interaction_only = True`
- Un escalado o una estandarización del dataframe resultante.

Desupués aplica esas transformaciones de testDF.

In [ ]:
#Utilizaremos california_housing_test.csv y lo dividiremos en trainDF y testDF

from sklearn.model_selection import train_test_split

df = pd.read_csv('/content/sample_data/california_housing_test.csv')

trainDF, testDF= train_test_split(df,test_size= 0.2)

###Ingeniería de características trainDF

In [ ]:
#Aumento de dimensionalidad con:
    # Características creadas por ti
        #1: Relación ingresos y valor casa
trainDF['income_house_value_ratio']= trainDF['median_income'] / trainDF['median_house_value']

        #2: Densidad de población
trainDF['population_density'] = trainDF['population'] / trainDF['households']

print(trainDF)

      longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
1954    -122.47     37.87                36.0       4471.0           618.0   
1602    -121.94     37.73                22.0       6719.0          1068.0   
104     -118.26     33.99                36.0       2016.0           505.0   
2909    -122.11     37.41                27.0       5110.0          1599.0   
538     -122.55     37.98                31.0       3807.0           828.0   
...         ...       ...                 ...          ...             ...   
2935    -117.97     34.05                33.0       1452.0           268.0   
697     -118.12     34.15                19.0        557.0           216.0   
2049    -117.30     34.10                44.0        589.0           130.0   
542     -118.30     33.88                29.0        850.0           229.0   
143     -120.06     36.95                24.0        646.0           134.0   

      population  households  median_income  median_house_value

In [ ]:
# Características polinómicas de orden 3, pero SOLO de 2 características que elijas y con la opción `interaction_only = True`

poly_columns= ['total_rooms', 'income_house_value_ratio']

trainDF_select= trainDF[poly_columns]


polyf2=PolynomialFeatures(degree=3, interaction_only=True)

polyf2.set_output(transform="pandas")

polyf2.fit(trainDF_select)
trainPolyDF= polyf2.transform(trainDF_select)

trainDF= pd.concat([trainDF, trainPolyDF.drop(columns=poly_columns)], axis=1)
print(trainDF)


      longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
1954    -122.47     37.87                36.0       4471.0           618.0   
1602    -121.94     37.73                22.0       6719.0          1068.0   
104     -118.26     33.99                36.0       2016.0           505.0   
2909    -122.11     37.41                27.0       5110.0          1599.0   
538     -122.55     37.98                31.0       3807.0           828.0   
...         ...       ...                 ...          ...             ...   
2935    -117.97     34.05                33.0       1452.0           268.0   
697     -118.12     34.15                19.0        557.0           216.0   
2049    -117.30     34.10                44.0        589.0           130.0   
542     -118.30     33.88                29.0        850.0           229.0   
143     -120.06     36.95                24.0        646.0           134.0   

      population  households  median_income  median_house_value

In [ ]:
#Estandarización
scalerStd2 = StandardScaler().set_output(transform="pandas")
scalerStd2.fit(trainDF) #Almacena media y desviación
scaleStd_trainDF = scalerStd2.transform(trainDF)

print("El DF quedaría así tras la estandarización:")
print(scaleStd_trainDF)

print("Esta es la información que se guarda:")
print_info(scaleStd_trainDF)

El DF quedaría así tras la estandarización:
      longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
1954      -1.43      1.03            5.65e-01         0.91            0.23   
1602      -1.16      0.96           -5.47e-01         1.98            1.34   
104        0.68     -0.79            5.65e-01        -0.27           -0.05   
2909      -1.25      0.81           -1.50e-01         1.21            2.66   
538       -1.47      1.08            1.68e-01         0.59            0.75   
...         ...       ...                 ...          ...             ...   
2935       0.83     -0.76            3.27e-01        -0.54           -0.64   
697        0.75     -0.71           -7.85e-01        -0.97           -0.77   
2049       1.17     -0.74            1.20e+00        -0.95           -0.98   
542        0.66     -0.84            9.23e-03        -0.83           -0.73   
143       -0.22      0.60           -3.88e-01        -0.93           -0.97   

      population  h

###Aplicar cambios a testDF




In [ ]:
print(trainDF.corr())



                                      longitude  latitude  housing_median_age  \
longitude                              1.00e+00 -9.24e-01               -0.05   
latitude                              -9.24e-01  1.00e+00               -0.03   
housing_median_age                    -5.32e-02 -3.30e-02                1.00   
total_rooms                            3.98e-02 -3.50e-02               -0.37   
total_bedrooms                         5.69e-02 -6.05e-02               -0.32   
population                             1.08e-01 -1.16e-01               -0.31   
households                             4.20e-02 -6.44e-02               -0.31   
median_income                         -2.74e-02 -6.84e-02               -0.16   
median_house_value                    -5.88e-02 -1.33e-01                0.08   
income_house_value_ratio              -2.78e-03  1.82e-01               -0.25   
population_density                     1.47e-02  1.07e-03                0.04   
1                           